# Band Structure for Twisted Molybdenum Disulfide bilayers at various angles.

## 0. Introduction.

This notebook demonstrates how to generate a twisted interface between two materials using commensurate lattices. The example uses molybdenum disulfide (MoS2) as both the film and substrate materials. The notebook uses the new `create_commensurate_interface` function which first creates a slab from the material and then performs commensurate lattice matching to find valid supercells for the target twist angle. The algorithm searches for supercell matrices within specified size limits to achieve the target twist angle within tolerance. The generated interface is visualized and analyzed to determine the actual twist angle and the number of atoms in the interface.
Then we calculate the band structure of the interface using a workflow, submitting a job and visualizing the results.

> **Kaihui Liu, Liming Zhang, Ting Cao, Chenhao Jin, Diana Qiu, Qin Zhou, Alex Zettl, Peidong Yang, Steve G. Louie & Feng Wang**
> Evolution of interlayer coupling in twisted molybdenum disulfide bilayers. Nature Communications, 5, 4966. 2014.
 > [https://doi.org/10.1038/ncomms5966](https://doi.org/10.1038/ncomms5966)

The twisted MoS2 bilayers are shown in the following Figure 4 from the article.

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/interfaces/twisted-bilayer-molybdenum-disulfide/MoS2-twisted-bilayers.png" alt="Twisted MoS2 bilayers" width="600"/>

## 1. Prepare the Environment
### 1.1. Set up the notebook
Let's set angles and corresponding distances for the twisted interface from the article.

In [ ]:
# Uncomment lines to reproduce specific cases from the article
INTERFACE_PARAMETERS = [
    # {"angle": 0.0, "distance": 6.8},
    # {"angle": 13.0, "distance": 6.5},
    # {"angle": 22.0, "distance": 6.5},
    # {"angle": 38.0, "distance": 6.5},
    # {"angle": 47.0, "distance": 6.5},
    {"angle": 60.0, "distance": 6.1},  # AB1
]

# Slab creation parameters
MILLER_INDICES = (0, 0, 1)  # Miller indices for slab creation
NUMBER_OF_LAYERS = 1  # Number of layers in the slab

INTERFACE_VACUUM = 20.0  # in Angstroms

# Search algorithm parameters
MAX_REPETITION = None  # Maximum supercell matrix element value (None for automatic)
ANGLE_TOLERANCE = 0.5  # in degrees
RETURN_FIRST_MATCH = True  # If True, returns first solution within tolerance

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
VISUALIZE_REPETITIONS = [3, 3, 1]

 ### 1.2. Install packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install("mat3ra-utils")
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|api_examples")

### 1.3. Get input material
We'll use the MoS2 material from Standata.


In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from utils.visualize import visualize_materials

material = Material.create(Materials.get_by_name_and_categories("MoS2", "2D"))

print("Initial material properties:")
print(f"Formula: {material.formula}")
print(f"Number of atoms: {len(material.basis.elements.ids)}")

if SHOW_INTERMEDIATE_STEPS:
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS)
    visualize_materials(material, repetitions=VISUALIZE_REPETITIONS, rotation="-90x")

 ## 3. Generate Twisted Interface
 ### 3.1. Create slab


In [ ]:
from mat3ra.made.tools.modify import translate_to_z_level
from mat3ra.esse.models.core.reusable.axis_enum import AxisEnum
from mat3ra.made.tools.helpers import create_slab
from mat3ra.made.tools.helpers import create_interface_commensurate as create_commensurate_interface

slab = create_slab(
    crystal=material,
    miller_indices=MILLER_INDICES,
    number_of_layers=NUMBER_OF_LAYERS,
    vacuum=0.0,  # No vacuum in the slab, it is a 2D material
)
slab = translate_to_z_level(slab, "center")

visualize_materials(slab, rotation="-90x")

### 3.2. Create twisted interfaces

In [ ]:
interfaces = []
for parameters in INTERFACE_PARAMETERS:
    interface = create_commensurate_interface(
        material=slab,
        target_angle=parameters["angle"],
        angle_tolerance=ANGLE_TOLERANCE,
        max_repetition_int=MAX_REPETITION,
        return_first_match=RETURN_FIRST_MATCH,
        direction=AxisEnum.z,
        gap=parameters["distance"],
        vacuum=INTERFACE_VACUUM,
    )

    # resetting labels to not interfere with pseudopotential element names
    original_labels = interface.basis.labels
    interface.basis.set_labels_from_list([])
    interface.name = f"{interface.name} {parameters['angle']} degrees"
    interfaces.append(interface)
    print(f"Created interface with twist angle {parameters['angle']}° and {len(interface.basis.elements.ids)} atoms")

## 4. Preview the  materials


In [ ]:
from utils.visualize import visualize_materials

for interface in interfaces:
    visualize_materials(interface, title=interface.name, viewer="wave")

## 5. Save materials

In [ ]:
from utils.jupyterlite import set_materials

set_materials(interfaces)

## 6. Add materials to your collection
### 6.1. Authenticate on the Platform

In [ ]:
ACCOUNT_ID = "ACCOUNT_ID"
AUTH_TOKEN = "AUTH_TOKEN"
ORGANIZATION_ID = "ORGANIZATION_ID"

import os
import sys

if sys.platform == "emscripten":
    # Only works if launched within the Platform, otherwise requires redirect to Login
    apiConfig = data_from_host.get("apiConfig")
    os.environ.update(data_from_host.get("environ", {}))
    os.environ.update(
        dict(
            ACCOUNT_ID=apiConfig.get("accountId"),
            AUTH_TOKEN=apiConfig.get("authToken"),
            MATERIALS_PROJECT_API_KEY=MATERIALS_PROJECT_API_KEY,
            ORGANIZATION_ID=apiConfig.get("organizationId") or "",
        )
    )

### 6.2. Add materials to your Collection via API


In [ ]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON
from exabyte_api_client.endpoints.materials import MaterialEndpoints

OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID
endpoint = MaterialEndpoints(*ENDPOINT_ARGS)

saved_materials = []
for interface in interfaces:
    RAW_CONFIG = interface.to_dict()
    fields = ["name", "lattice", "basis"]
    CONFIG = {key: RAW_CONFIG[key] for key in fields}

    saved_material = endpoint.create(CONFIG, OWNER_ID)
    saved_materials.append(saved_material)

### 6.3. Verify material creation

In [ ]:
for saved_material in saved_materials:
    display_JSON(saved_material)

### 6.4. Get material id

In [ ]:
material_ids = [saved_material.get("_id") for saved_material in saved_materials]
print("Material IDs:", material_ids)

## 7. Setup workflow

### 7.1. Set Parameters

- **QUERY**: A query describing the documents to find. See [Meteor collection](https://docs.meteor.com/api/collections.html#Mongo-Collection-find) for more information.

- **limit**: Maximum number of results to return. See [Meteor collection](https://docs.meteor.com/api/collections.html#Mongo-Collection-find) for more information.

In [ ]:
OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID
WORKFLOW_SYSTEM_NAME = "espresso-band-structure"  # Name of the workflow to copy
RELAXATION_WORKFLOW_QUERY = {"systemName": "espresso-variable-cell-relaxation"}

OPTIONS = {"limit": 1}

### 7.2. Initialize the endpoint

Initialize a helper class to interact with `WorkflowEndpoints`. This only has to be done once.

In [ ]:
from exabyte_api_client import BankWorkflowEndpoints, WorkflowEndpoints
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON, copy_bank_workflow_by_system_name

bank_workflow_endpoints = BankWorkflowEndpoints(*ENDPOINT_ARGS)
workflow_endpoints = WorkflowEndpoints(*ENDPOINT_ARGS)

band_structure_workflow_id = copy_bank_workflow_by_system_name(bank_workflow_endpoints, WORKFLOW_SYSTEM_NAME, OWNER_ID)

### 7.3. Get the relaxation subworkflow

In [ ]:
relaxation_workflow = bank_workflow_endpoints.list(RELAXATION_WORKFLOW_QUERY, OPTIONS)[0]
swf_0 = relaxation_workflow["subworkflows"][0]

### 7.4. Create the modifier for the workflow

Contact the endpoint to get the workflow by its ID. Adjust the necessary parameters in the workflow modifier to set the functional and method for the band gap calculation.

In [ ]:
QUERY = {"_id": band_structure_workflow_id, "owner._id": OWNER_ID}
workflow = workflow_endpoints.list(QUERY, OPTIONS)[0]

modifier_0 = {
    "name": "Band Structure (LDA)",
}

swf_1 = workflow["subworkflows"][0]

model = {
    "type": "dft",
    "subtype": "lda",  # Is changed
    "method": {"type": "pseudopotential", "subtype": "us", "data": {}},
    "functional": {"slug": "pz"},  # Is changed
    "refiners": [],
    "modifiers": [],
}

### 7.5.Combine the subworkflows

In [ ]:
swf_0["model"] = model
swf_1["model"] = model

modifier_1 = {"subworkflows": [swf_0, swf_1]}

### 7.6. Update workflow



In [ ]:
workflow_endpoints.update(id_=band_structure_workflow_id, modifier=modifier_0)
workflow_endpoints.update(id_=band_structure_workflow_id, modifier=modifier_1)

### 7.7. Get units

In [ ]:
unit_0 = relaxation_workflow["units"][0]
unit_1 = workflow["units"][0]

# Connect the units
unit_0["next"] = unit_1["flowchartId"]
unit_1["head"] = False

### 7.8. Create modifier for units

In [ ]:
modifier_2 = {"units": [unit_0, unit_1]}
workflow_endpoints.update(id_=band_structure_workflow_id, modifier=modifier_2)

### 7.9. Verify the workflow

In [ ]:
# Fetch the updated workflow
workflow = workflow_endpoints.list(QUERY, OPTIONS)[0]
display_JSON(workflow)

## 8. Create a job with the workflow and material
### 8.1. Set Parameters

In [ ]:
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.jobs import JobEndpoints
from exabyte_api_client.endpoints.projects import ProjectEndpoints

from datetime import datetime

material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
OWNER_ID = os.getenv("ORGANIZATION_ID") or ACCOUNT_ID

project_endpoints = ProjectEndpoints(*ENDPOINT_ARGS)
project_id = project_endpoints.list({"isDefault": True, "owner._id": OWNER_ID})[0]["_id"]

job_endpoints = JobEndpoints(*ENDPOINT_ARGS)

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")  # for name to be found easily

JOB_NAME = f"Band Structure {timestamp}"

### 8.2. Create jobs

In [ ]:
jobs = job_endpoints.create_by_ids(
    materials=saved_materials, workflow_id=workflow["_id"], project_id=project_id, prefix=JOB_NAME, owner_id=OWNER_ID
)

display_JSON(jobs)

### 8.3. Move jobs to set

In [ ]:
from exabyte_api_client import ProjectEndpoints

project_endpoints = ProjectEndpoints(*ENDPOINT_ARGS)
project_id = project_endpoints.list({"isDefault": True, "owner._id": OWNER_ID})[0]["_id"]

JOBS_SET_NAME = None

if JOBS_SET_NAME is not None:
    jobs_set = job_endpoints.create_set({"name": JOBS_SET_NAME, "projectId": project_id, "owner": {"_id": OWNER_ID}})
    for job in jobs:
        job_endpoints.move_to_set(job["_id"], "", jobs_set["_id"])

workflow_in_job = jobs[0]["workflow"]

### 8.2. Update job with new parameters

In [ ]:
kgrid_relax = [6, 6, 1]

In [ ]:
modifier_relax = {
    "workflow.subworkflows.0.units.0.context": {
        "kgrid": {
            "dimensions": kgrid_relax,
            "shifts": [0, 0, 0],
            "reciprocalVectorRatios": [1, 1, 1],
            "gridMetricType": "KPPRA",
            "gridMetricValue": 2,
            "preferGridMetric": False,
        },
        "isKgridEdited": True,
        "subworkflowContext": {},
    }
}

### 8.2.1. Set `rendered` field

In [ ]:
import re

rendered_relax_str = workflow_in_job["subworkflows"][0]["units"][0]["input"][0]["rendered"]
print(rendered_relax_str)
"workflow.subworkflows.0.units.0.input.0.rendered"

modifier_relax["workflow.subworkflows.0.units.0.input.0.rendered"] = re.sub(
    r"K_POINTS automatic\n(\d+) (\d+) (\d+) (\d+) (\d+) (\d+)",
    f"K_POINTS automatic\n{kgrid_relax[0]} {kgrid_relax[1]} {kgrid_relax[2]} 0 0 0",
    rendered_relax_str,
)

### 8.3. Update each job with the modifier

In [ ]:
for job in jobs:
    job_endpoints.update(id_=job["_id"], modifier=modifier_relax)

## 9. Submit the job

In [ ]:
for job in jobs:
    job_endpoints.submit(job["_id"])

## 10. Get results
### 10.1. Wait for jobs to finish

In [ ]:
from utils.generic import wait_for_jobs_to_finish

job_ids = [job["_id"] for job in jobs]

wait_for_jobs_to_finish(job_endpoints, job_ids, poll_interval=60)

### 10.2. Get the Band Gap property

In [ ]:
from utils.generic import get_property_by_subworkflow_and_unit_indicies
from exabyte_api_client.endpoints.properties import PropertiesEndpoints

property_endpoints = PropertiesEndpoints(*ENDPOINT_ARGS)

results = []
for material in saved_materials:
    job = next((job for job in jobs if job["_material"]["_id"] == material["_id"]))
    unit_flowchart_id = job["workflow"]["subworkflows"][0]["units"][1]["flowchartId"]
    band_structure = property_endpoints.get_property(
        job["_id"],
        unit_flowchart_id=unit_flowchart_id,
        property_name="band_structure",
    )

    results.append(
        {
            "material_id": material["_id"],
            "band_structure": band_structure,
        }
    )

### 10.3. Plotting functions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def get_high_symmetry_points():
    """Common high-symmetry points for hexagonal lattices (like MoS2)"""
    return {
        (0, 0, 0): "Γ",
        (0.5, 0, 0): "M",
        (1 / 3, 1 / 3, 0): "K",
        (0, 0, 0.5): "A",
        (0.5, 0, 0.5): "L",
        (1 / 3, 1 / 3, 0.5): "H",
    }


def find_high_symmetry_labels(xDataArray, tolerance=1e-3):
    """Find high-symmetry points in the k-path"""
    high_sym_points = get_high_symmetry_points()
    labels = []
    positions = []

    for i, k_point in enumerate(xDataArray):
        k_frac = np.array(k_point)
        for sym_k, label in high_sym_points.items():
            if np.allclose(k_frac, sym_k, atol=tolerance):
                labels.append(label)
                positions.append(i)
                break

    return positions, labels


def plot_band_structure_with_labels(band_structure_data, ylim=None, title="Band Structure"):
    xDataArray = band_structure_data["xDataArray"]
    yDataSeries = band_structure_data["yDataSeries"]

    # Calculate k-point distances
    k_distances = [0]
    for i in range(1, len(xDataArray)):
        prev_k = np.array(xDataArray[i - 1])
        curr_k = np.array(xDataArray[i])
        distance = np.linalg.norm(curr_k - prev_k)
        k_distances.append(k_distances[-1] + distance)

    # Find high-symmetry points
    sym_positions, sym_labels = find_high_symmetry_labels(xDataArray)

    # Plot bands
    plt.figure(figsize=(10, 6))
    for energies in yDataSeries:
        plt.plot(k_distances, energies, "b-", linewidth=1)

    # Add vertical lines at high-symmetry points
    for pos in sym_positions:
        plt.axvline(x=k_distances[pos], color="r", linestyle="--", alpha=0.5)

    # Set x-axis labels
    if sym_positions and sym_labels:
        plt.xticks([k_distances[pos] for pos in sym_positions], sym_labels)

    # Set y-axis limits
    if ylim:
        plt.ylim(ylim)

    plt.xlabel("k-point path")
    plt.ylabel("Energy (eV)")
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

### 10.4. Plot the band structure

In [ ]:
MAX_E = 4.0  # Maximum energy for y-axis limit, eV
MIN_E = -2.0  # Minimum energy for y-axis limit, eV

for result in results:
    if result["band_structure"]:
        band_data = result["band_structure"]["data"]
        plot_band_structure_with_labels(band_data, ylim=[MIN_E, MAX_E])